In [1]:
from simulator.bb84 import simulation_bb84
from tqdm import tqdm
from numpy import arange

In [2]:
results_list = []
for qber in tqdm(arange(start=0.01, stop=0.16, step = 0.01)):
    simulation_results = simulation_bb84(
        alice_basis_length=500,
        disturbance_probability=qber,
        cascade_n_passes=4
    )
    results_list.append(simulation_results)
    # How long will it take to perform this simple test?

100%|██████████| 15/15 [00:00<00:00, 17.39it/s]


In [3]:
for block_set in results_list[-1].get('cascade history'):
    alice_blocks = block_set.get('Alice blocks')
    bob_blocks = block_set.get('Bob blocks')
    for i in range(len(bob_blocks)):
        alice_keys = list(alice_blocks[i].keys())
        bob_keys = list(bob_blocks[i].keys())
        if '161' in bob_keys:
            print("\nNew instance of 161 key found.")
            print(alice_keys)
            print(bob_keys)


New instance of 161 key found.
['161', '46', '7', '132', '171', '92']
['161', '46', '7', '132', '171', '92']

New instance of 161 key found.
['84', '45', '75', '60', '152', '172', '161', '164', '148', '109', '86', '203']
['84', '45', '75', '60', '152', '172', '161', '164', '148', '109', '86', '203']

New instance of 161 key found.
['66', '146', '34', '15', '193', '11', '173', '170', '108', '44', '59', '148', '102', '107', '140', '178', '71', '50', '0', '134', '196', '190', '161', '131']
['66', '146', '34', '15', '193', '11', '173', '170', '108', '44', '59', '148', '102', '107', '140', '178', '71', '50', '0', '134', '196', '190', '161', '131']

New instance of 161 key found.
['154', '204', '42', '4', '188', '46', '48', '55', '50', '58', '63', '10', '113', '140', '20', '51', '111', '93', '44', '183', '75', '96', '161', '117', '26', '202', '163', '91', '53', '54', '116', '29', '103', '118', '166', '156', '170', '88', '43', '187', '79', '143', '160', '123', '39', '152', '150', '83']
['154